# 5.0 A very brief intro to MNE



MNE is an open-source software package to help you analyze and visualize M/EEG, ECoG, and other types of neurophysiological data. Things you can do in MNE include:

* visualizing data
* epoching
* averaging and computing evoked responses
* computing ICA 
* time-frequency analysis
* computing connectivity estimates
* classifier analysis 
* non-parametric stats

...and many more. There's a wealth of documentation (https://mne.tools/stable/overview/index.html) and examples (https://mne.tools/stable/auto_examples/index.html) online. 

In today's session, we'll briefly walk you through one of the MNE introductory examples so you can get a general feel for how to use MNE. 

Here's the original example that we'll be using: https://mne.tools/stable/auto_tutorials/intro/plot_10_overview.html#sphx-glr-auto-tutorials-intro-plot-10-overview-py. We'll plot evoked responses today, but the original example also shows you how to run and plot a time-frequency analysis and a bunch of other cool things. 

## 5.1 Installing MNE

Downloading and installing MNE can take a little time, so for the purpose of today's session (or if you're generally not that interested in analyzing M/EEG data), we'd advise you to just follow along with the lecture from here on out, rather than downloading and installing the package and example data set.

Installation instructions can be found here: https://mne.tools/stable/install/index.html. 


## 5.2 Reading in raw data

Once you have MNE installed, you can already read in your raw data! MNE supports several different data formats, including fif, brainvision, eeglab, and edf files. You can read more about that here: https://martinos.org/mne/stable/manual/io.html

The example we're using works with MNE's sample data set, which comes as fif.

In [ ]:
import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_filt-0-40_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)

Let's have a look at the data structure:

In [ ]:
print(raw)

In [ ]:
print(raw.info)

This gives us a lot of useful information about our raw data: We can see that there are 376 channels (chs and nchan), what they're called (ch_names), the events (0 at this point, because we haven't defined them yet), channels marked as bad (MEG 2443 and EEG 053, apparently), highpass and lowpass filters that have been applied (0.1 Hz and 40 Hz), the sampling frequency (sfreq), and a lot more. We can look at each of these in more detail:

In [ ]:
# for example, look at the channel names
print(raw.ch_names)

In [ ]:
# or look at the bad channels
print(raw.info['bads'])

## 5.3 Browsing raw data

Now let's have a look at a segment of our data:

In [ ]:
'''Note that usually, when using MNE outside of jupyter notebooks, 
this will open an interactive window where you can scroll through your data 
and, for example, mark bad channels manually.

For now, we'll just show a preview of what that looks like.'''

raw.plot_psd(fmax=50) # plot up to 50 Hz
raw.plot()

You can read more about how to browse raw data in MNE here: https://mne.tools/stable/auto_tutorials/raw/plot_40_visualize_raw.html?highlight=browse

## 5.4 Preprocessing / ICA

You can use Independent Component Analysis (ICA) in mne. In the code snippet below (from the mne documentation), it is assumed that you already know which components to inspect (i.e., components 1 and 2). Usually, you'll have to take a few more steps to determine which components capture the artifacts best (https://mne.tools/stable/auto_tutorials/preprocessing/plot_40_artifact_correction_ica.html#tut-artifact-ica).

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
ica.exclude = [1, 2]  # details on how we picked these are omitted here
ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
# back-project ICA to the raw signal

orig_raw = raw.copy()
raw.load_data()
ica.apply(raw)

## 5.4 Epoching data

MNE can extract events for you like this:

In [ ]:
# use function .find_events on the raw data
# and tell MNE that your stimulus channel is called 'STI 014'
events = mne.find_events(raw, stim_channel='STI 014')

# look at the first five events
print(events[:5])

In [ ]:
# recode events for clarity

event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
              'visual/right': 4, 'smiley': 5, 'buttonpress': 32}

If you'd like to exclude some noisy channels, you can save all the bad channels in a list, pass that list to MNE, and select only the remaining good channels, like this:

In [ ]:
raw.info['bads'] += ['MEG 2443', 'EEG 053']
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True, stim=False,
                       exclude='bads')

Now the data frame 'picks' includes all the MEG, EEG and EOG channels, excluding the ones that are in your list of 'bad' channels and the STIM channels.

In [ ]:
# define parameters for epoching

# first, define rejection criteria
reject_criteria = dict(mag=4000e-15,     # 4000 fT
                       grad=4000e-13,    # 4000 fT/cm
                       eeg=150e-6,       # 150 μV
                       eog=250e-6)       # 250 μV

# epoch the data
epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-0.2, tmax=0.5,
                    reject=reject_criteria, preload=True)
print(epochs)

As you can see above, the epochs data frame now includes different conditions: 'auditory/left', 'visual/left', and so on. mne also gives you a summary of how many trials there are per condition. In a next step, you could pool across left/right stimulus presentation to compare auditory to visual trials: 

In [ ]:
conds_we_care_about = ['auditory/left', 'auditory/right',
                       'visual/left', 'visual/right']
epochs.equalize_event_counts(conds_we_care_about)  # this operates in-place
aud_epochs = epochs['auditory']
vis_epochs = epochs['visual']
del raw, epochs  # free up memory

And now you could plot evoked responses for auditory and visual responses: 

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

mne.viz.plot_compare_evokeds(dict(auditory=aud_evoked, visual=vis_evoked),
                             legend='upper left', show_sensors='upper right')

There really is .A LOT. more you can do with MNE, but the techniques you'll be using obviously depend a lot on your research question and data. If you're interested in using MNE for data analysis, we can't recommend their documentation, tutorials, and example gallery enough! (See links in section X.0). Have fun! :) 